# Test BERT-Pytorch

BERT-pytorch is a PyTorch implementation of the BERT algorithm.

[BERT-pytorch](https://github.com/codertimo/BERT-pytorch)


## Embedding
In the BERT implemetnation (bert_pytorch/model/bert.py), the masking is done after the second token (x>0) since in the original BERT paper, the first element of the input is always \[CLS\]. In our model, we will use the variant name as the \[CLS\] and the values are:
[wt, alpha, delta, omicron, na], where "na" stands for not assigned.

In [3]:
import io
import copy
import math
from Bio import SeqIO
import torch
import torch.nn as nn
# from bert_pytorch.model import BERT

## Tokenization and Vocabulary
In [ProteinBERT](https://academic.oup.com/bioinformatics/article/38/8/2102/6502274), Brandes et al used 26 unique tokens to represent the 20 standard amino acids, selenocysteine (U), and undefined amino acid (X), another amino acid (OTHER) and three speical tokens \<START\>, \<END\>, \<PAD\>.

In [4]:
# Based on the source code of protein_bert
ALL_AAS = 'ACDEFGHIKLMNPQRSTUVWXY'
ADDITIONAL_TOKENS = ['<OTHER>', '<START>', '<END>', '<PAD>']

# Each sequence is added <START> a
ADDED_TOKENS_PER_SEQ = 2

n_aas = len(ALL_AAS)
aa_to_token_index = {aa: i for i, aa in enumerate(ALL_AAS)}
additional_token_to_index = {token: i + n_aas for i, token in enumerate(ADDITIONAL_TOKENS)}
token_to_index = {**aa_to_token_index, **additional_token_to_index}
index_to_token = {index: token for token, index in token_to_index.items()}
n_tokens = len(token_to_index)

def tokenize_seq(seq):
    other_token_index = additional_token_to_index['<OTHER>']
    return [additional_token_to_index['<START>']] + [aa_to_token_index.get(aa, other_token_index) for aa in seq] + [additional_token_to_index['<END>']]

## Amino Acid Token Embeddings
We will derive it from the [torch.nn.Embedding class](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html). The size of the vacabulary equals the number of tokens. This approach allows the learning of the embeddings from the model intself. If we train the model with virus sepcific squences, the embeddings shall reflect the hidden properties of the amino acids in context of the trainign sequences. Note that the \<START\> and \<END\> tokens are always added at the beginning of the sequence. \<PAD\> tokens may be added before the \<END\> token if the sequence is shorter than the input sequence.

Note that using the "from_pretrained" class method of torch.nn.Embedding, we can load pretrained weights of the embedding.


In [5]:
class TokenEmbedding(nn.Embedding):
    def __init__(self, num_embeddings: torch.Tensor, embedding_dim: int = 512, padding_idx=None):
        super().__init__(num_embeddings, embedding_dim, padding_idx)

padding_idx = token_to_index['<PAD>']
print(padding_idx)


25


In [6]:
test_wt_seq = """>sp|P0DTC2|SPIKE_SARS2 Spike glycoprotein OS=Severe acute respiratory syndrome coronavirus 2 OX=2697049 GN=S PE=1 SV=1
MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFS
NVTWFHAIHVSGTNGTKRFDNPVLPFNDGVYFASTEKSNIIRGWIFGTTLDSKTQSLLIV
NNATNVVIKVCEFQFCNDPFLGVYYHKNNKSWMESEFRVYSSANNCTFEYVSQPFLMDLE
GKQGNFKNLREFVFKNIDGYFKIYSKHTPINLVRDLPQGFSALEPLVDLPIGINITRFQT
LLALHRSYLTPGDSSSGWTAGAAAYYVGYLQPRTFLLKYNENGTITDAVDCALDPLSETK
CTLKSFTVEKGIYQTSNFRVQPTESIVRFPNITNLCPFGEVFNATRFASVYAWNRKRISN
CVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIAD
YNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPC
NGVEGFNCYFPLQSYGFQPTNGVGYQPYRVVVLSFELLHAPATVCGPKKSTNLVKNKCVN
FNFNGLTGTGVLTESNKKFLPFQQFGRDIADTTDAVRDPQTLEILDITPCSFGGVSVITP
GTNTSNQVAVLYQDVNCTEVPVAIHADQLTPTWRVYSTGSNVFQTRAGCLIGAEHVNNSY
ECDIPIGAGICASYQTQTNSPRRARSVASQSIIAYTMSLGAENSVAYSNNSIAIPTNFTI
SVTTEILPVSMTKTSVDCTMYICGDSTECSNLLLQYGSFCTQLNRALTGIAVEQDKNTQE
VFAQVKQIYKTPPIKDFGGFNFSQILPDPSKPSKRSFIEDLLFNKVTLADAGFIKQYGDC
LGDIAARDLICAQKFNGLTVLPPLLTDEMIAQYTSALLAGTITSGWTFGAGAALQIPFAM
QMAYRFNGIGVTQNVLYENQKLIANQFNSAIGKIQDSLSSTASALGKLQDVVNQNAQALN
TLVKQLSSNFGAISSVLNDILSRLDKVEAEVQIDRLITGRLQSLQTYVTQQLIRAAEIRA
SANLAATKMSECVLGQSKRVDFCGKGYHLMSFPQSAPHGVVFLHVTYVPAQEKNFTTAPA
ICHDGKAHFPREGVFVSNGTHWFVTQRNFYEPQIITTDNTFVSGNCDVVIGIVNNTVYDP
LQPELDSFKEELDKYFKNHTSPDVDLGDISGINASVVNIQKEIDRLNEVAKNLNESLIDL
QELGKYEQYIKWPWYIWLGFIAGLIAIVMVTIMLCCMTSCCSCLKGCCSCGSCCKFDEDD
SEPVLKGVKLHYT"""
len(test_wt_seq)

1413

In [7]:
test_seqs = []
fa_parser = SeqIO.parse(io.StringIO(test_wt_seq), 'fasta')
for record in fa_parser:
    seq = record.seq
    test_seqs.append(str(seq))

In [8]:
num_embeddings = n_tokens
embedding_dim = 20
embedding = TokenEmbedding(num_embeddings, embedding_dim, padding_idx)
test_embedding = embedding(torch.IntTensor(tokenize_seq(test_seqs[0])))
print(test_embedding)
print(f'Shape of test sequence embedding: {test_embedding.shape}')


tensor([[-0.9295, -0.3930, -1.6340,  ...,  1.0091,  0.8342,  0.6325],
        [ 1.5217,  2.7205, -0.4793,  ..., -0.9853,  1.0511,  1.3045],
        [ 0.0953,  1.9266,  0.0882,  ..., -0.8576, -0.1565,  0.6572],
        ...,
        [-0.2898,  0.1287,  1.8492,  ...,  0.6741,  1.0530,  0.2916],
        [ 0.9037, -0.3816,  0.4129,  ...,  0.0431, -0.4507, -0.2156],
        [-0.2640, -0.2618,  1.2745,  ...,  2.4348,  1.5389, -2.0573]],
       grad_fn=<EmbeddingBackward0>)
Shape of test sequence embedding: torch.Size([1275, 20])


Let's take a look of the embedding weights:

In [9]:
embedding.weight

Parameter containing:
tensor([[ 0.1655, -1.8507,  2.3808,  1.7079, -0.8344,  0.0397, -1.1526, -0.7542,
         -0.3881,  2.5951,  0.0365, -1.1324,  2.1356,  0.6057, -0.4333, -0.2600,
         -2.1362,  0.0072,  1.5041, -0.8161],
        [ 1.3647,  0.4286, -0.0438, -0.0751,  0.3926, -0.2939,  1.0673, -0.7275,
          0.4386,  0.5220,  0.3631,  0.2311,  0.9486,  0.3194, -2.0667,  1.0608,
          0.3539, -0.9689, -0.6506, -0.4407],
        [-0.5890, -0.9713,  0.4251, -0.7520,  1.7415, -0.1838,  0.4673, -1.9181,
          0.4152, -0.2488, -0.3671, -1.4407,  1.4268,  1.8225,  0.2309,  0.1728,
         -0.5015,  0.7601,  0.2511,  1.2167],
        [-0.9015,  0.8305, -1.2921, -0.8276,  1.1848, -0.3986,  1.0509,  0.6731,
          0.2016, -0.0459, -1.0505,  0.2374,  0.4629,  0.9688, -1.7671,  0.5760,
          0.7361, -1.4768,  0.2024,  0.2315],
        [ 0.0953,  1.9266,  0.0882, -1.2296,  0.8251, -0.1247, -0.4404, -1.1608,
         -1.7103,  0.1154, -0.3154, -0.6004,  0.6913,  1.3998,  0

In [10]:
embedding.weight.shape

torch.Size([26, 20])

## Postional Encoding
We will use the  sine and cosine functions of different frequencie to embed positional information as in the original BERT method.

In [26]:
class PositionalEncoding(nn.Module):
    """
    Impement the PE function.
    
    The PE forward function is different from the BERT-pytorch. Here we used the original method in BERT so
    PE embeddings are added to the input embeddings and no graident tracking is used.
    """

    def __init__(self, d_model, dropout, max_len=1275):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * -(math.log(10000.0)/d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer("pe", pe)

    def forward(self, x):
        print(f'x.shape in PositionalEncoding: {x.shape}')
        print(f'x.shape: {x.shape},pe.shape: {self.pe.shape}')
        print(f'pe[:, : x.size(1)]: {self.pe[:, : x.size(1)].shape}')


        x = x + self.pe[:, : x.size(1)].requires_grad_(False)
        return self.dropout(x)

In [27]:
class SeqEncoding(nn.Module):
    """
    Encode amino acid sequence. Input sequence is represented by summing the corresponding sequence token,
    segment (e.g. question and answer or any segments separated by <SEP>), and position embeddings. In our 
    model, we only need the token and position embedding so segment embeddign is not implemented here.    
    """
    def __init__(self, num_embeddings, embedding_dim, dropout=0.1):
        super().__init__()
        self.token_embedding = TokenEmbedding(num_embeddings, embedding_dim)
        self.position = PositionalEncoding(embedding_dim, dropout)
        self.embeddng_dim = embedding_dim
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, seq:str):
        x = torch.IntTensor(tokenize_seq(seq))
        x = self.token_embedding(x)
        x = self.position(x)
        return self.dropout(x)

In [28]:
test_seq_encode = SeqEncoding(n_tokens, 512, 0.1)
num_parameters_seq_encoding = sum(p.numel() for p in test_seq_encode.parameters() if p.requires_grad)
print(f'Parameters in SeqEncoding: {num_parameters_seq_encoding}')
print(test_seq_encode(test_seqs[0]))

Parameters in SeqEncoding: 13312
x.shape in PositionalEncoding: torch.Size([1275, 512])
x.shape: torch.Size([1275, 512]),pe.shape: torch.Size([1275, 512])
pe[:, : x.size(1)]: torch.Size([1275, 512])
tensor([[ 0.2236,  0.0085,  0.3063,  ..., -0.2313, -0.6815, -0.6398],
        [-0.1819,  2.5132, -0.0000,  ..., -0.5560,  0.0000, -1.1247],
        [ 3.0274, -1.8674,  1.9210,  ..., -0.2102, -0.0000,  1.2327],
        ...,
        [ 0.5832, -1.2684, -0.4853,  ...,  0.0000, -0.0000, -1.2761],
        [-0.7092, -0.0000,  0.5969,  ...,  3.0542,  0.5780,  1.5972],
        [-0.0951, -1.1107, -0.3251,  ...,  1.2578,  0.0000,  3.8438]],
       grad_fn=<MulBackward0>)


# Model Definition

Here we define a model based on BERT. Part of the implementation is based on [BERT-pytorch](https://github.com/codertimo/BERT-pytorch)

In [30]:
def clones(module, N):
    """Produce N identical layers"""
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

In [ ]:
class Transformer:
    

In [ ]:
class BERT(nn.Module):
    """
    BERT model
    """

    def __init__(self, vocabl_size: int = 26, hidden: int = 768, n_layer: int = 12, attn_heads: int = 12, dropout: float = 0.1):
        """
        vacab_size: vacabulary or token size
        hidden: BERT model size (used as input size and hidden size)
        n_layer: number of Transformer layers
        attn_heads: attenion heads
        dropout: dropout ratio
        """

        super().__init__()
        self.hidden  = hidden
        self.n_layer = n_layer
        self.attn_heads = attn_heads

        self.feed_forward_hidden = hidden * 4
        self.embedding = TokenEmbedding(vocabl_size, embed_size=hidden, padding_idx=25)

        self.transformer_blocks = clones(Transformer(hidden, attn_heads, hidden *4, dropout), n_layer)

    def forward(self)
